# <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and Google Colab</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Sunday, October 17th submitted via Canvas by 11:59 pm</p>

This will be a in-class project done in teams of 2. 

In this Project, we will work with CIFAR10 image dataset. 
The starter code to download the database using keras is given below. 
Test the project on Google Colab running on a CPU, GPU and TPU
 

# In every line of code, please write a comment to briefly explain what that line is doing.
Your grades will be based on your understanding of the code you write! 


# Task 1
Convert the features in a form that can be given as input to tensorflow library/functions

In this task you will perform data augmentation. That is, pre-process the data to make the model more robust. Experiment with data augmentation techniques like rotation, translation, horizontal-flip, scaling, ZCA whitening and histogram equalization. 
You can choose any two or more augmentation technique(s) of your choice. 

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint


In [3]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
y_test.shape

(10000, 1)

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 1)

In [4]:
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


In [ ]:
## Data Augmentation 


In [ ]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

# Task 2
Try to build a Neural Network model, train on the features and report the accuracy.
Report your observations on the time taken on CPU and GPU (with and without CuDNN kernel) 



1.   Create a CNN based model with 4 hidden layers with 64, 128, 256 and 512 units in each succesive layer. Use a 5x5 convolution kernel and change as necessary. (Use at least 2 augmentations on your input) 
2.   Create an LSTM based model with 1 LSTM layer with 256 units. 



In [5]:
##CNN Model

def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(512, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

In [6]:
with tf.device('/device:GPU:0'):
  model = create_model()
  model.compile(
      
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])


In [ ]:
%%time

print('Using real-time data augmentation.')
print("CNN with GPU")
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = True
num_classes = 10


# Prepare model model saving directory.
filepath = 'cifar10.h5'

# Prepare callbacks for model saving and for learning rate adjustment.
'''checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
callbacks = [checkpoint]'''
with tf.device('/device:GPU:0'):
  # This will do preprocessing and realtime data augmentation:
  datagen = ImageDataGenerator(
      
      # set input mean to 0 over the dataset
      featurewise_center=False,
      # set each sample mean to 0
      samplewise_center=False,
      # divide inputs by std of dataset
      featurewise_std_normalization=False,
      # divide each input by its std
      samplewise_std_normalization=False,
      # apply ZCA whitening
      zca_whitening=True,
      # epsilon for ZCA whitening
      zca_epsilon=1e-06,
      # randomly rotate images in the range (deg 0 to 180)
      rotation_range=180,
      # randomly shift images horizontally
      width_shift_range=0.1,
      # randomly shift images vertically
      height_shift_range=0.1,
      # set range for random shear
      shear_range=0.2,
      # set range for random zoom
      zoom_range=0.2,
      # set range for random channel shifts
      #channel_shift_range=0.,
      # set mode for filling points outside the input boundaries
      fill_mode='nearest',
      # value used for fill_mode = "constant"
      cval=0.,
      # randomly flip images
      horizontal_flip=True,
      # randomly flip images
      vertical_flip=False,
      # set rescaling factor (applied before any other transformation)
      rescale=None,
      # set function that will be applied on each input
      preprocessing_function=None,
      # image data format, either "channels_first" or "channels_last"
      data_format=None,
      # fraction of images reserved for validation (strictly between 0 and 1)
      validation_split=0.0)

  # Compute quantities required for featurewise normalization
  # (std, mean, and principal components if ZCA whitening is applied).

  datagen.fit(x_train)

  
  import time
  print("CNN GPU")
  start = time.time()

  # Fit the model on the batches generated by datagen.flow().
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                      validation_data=(x_test, y_test),
                      epochs=epochs, verbose=1, workers=4
                      )
  stop = time.time()
  print(f"Training time: {stop - start}s")


Using real-time data augmentation.
CNN with GPU


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


CNN GPU
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1563/1563 [==============================] - 210s 134ms/step - loss: 1.2940 - sparse_categorical_accuracy: 0.5456 - val_loss: 5.7156 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/100
1563/1563 [==============================] - 217s 139ms/step - loss: 1.2681 - sparse_categorical_accuracy: 0.5560 - val_loss: 3.5852 - val_sparse_categorical_accuracy: 0.0810
Epoch 3/100
1563/1563 [==============================] - 221s 141ms/step - loss: 1.2454 - sparse_categorical_accuracy: 0.5656 - val_loss: 6.2714 - val_sparse_categorical_accuracy: 0.1006
Epoch 4/100
1563/1563 [==============================] - 215s 137ms/step - loss: 1.2363 - sparse_categorical_accuracy: 0.5704 - val_loss: 4.2314 - val_sparse_categorical_accuracy: 0.1002
Epoch 5/100
1563/1563 [==============================] - 217s 138ms/step - loss: 1.2210 - sparse_categorical_accuracy: 0.5746 - val_loss: 4.3091 - val_sparse_categorical_accuracy: 0.0987
Epoch 6/100
1563/1563 [==============================] - 218s 139ms/step - lo

CNN On CPU

In [ ]:
model_cpu = create_model()
model_cpu.compile(
    
      
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

In [ ]:
%%time
print('Using real-time data augmentation.')
print("CNN ON CPU")
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 10
data_augmentation = True
num_classes = 10


# Prepare model model saving directory.
filepath = 'cifar10.h5'

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
callbacks = [checkpoint]
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    
      
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=True,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=180,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.2,
    # set range for random zoom
    zoom_range=0.2,
    # set range for random channel shifts
    #channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).

datagen.fit(x_train)
import time

start = time.time()

print("CNN on CPU")

# Fit the model on the batches generated by datagen.flow().
model_cpu.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1, workers=4,
                    callbacks=callbacks)
stop = time.time()
print(f"Training time CPU: {stop - start}s")


Using real-time data augmentation.
CNN ON CPU


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


CNN on CPU


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 628/1563 [===========>..................] - ETA: 15:55 - loss: 2.7716 - sparse_categorical_accuracy: 0.1718

### Build a new model with CuDNN kernel


In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
model.save_weights('./cifar.h5', overwrite=True)

# Task 3
Run the LSTM solution in Task2 on a TPU and report the performance 